## Deliverable 3. Create a Travel Itinerary Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
clean_vacation_df = vacation_df.dropna()
clean_vacation_df.head(10)

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,0,Buala,SB,77.41,overcast clouds,-8.1450,159.5921,Maringe Lagoon Lodge
1,4,Atuona,PF,78.64,few clouds,-9.8000,-139.0333,Villa Enata
2,14,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,Paradise Inn
3,22,Olinda,BR,83.93,broken clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda
5,28,Manaure,CO,88.05,overcast clouds,11.7751,-72.4445,Hotel Palaaima
6,32,Arlit,NE,87.93,overcast clouds,18.7369,7.3853,Case de Passage De la Prefecture
7,35,Rikitea,PF,75.60,scattered clouds,-23.1203,-134.9692,People ThankYou
8,36,Miraflores,MX,84.02,broken clouds,19.2167,-98.8083,Hotel Mansión
9,38,Fernandina Beach,US,85.01,clear sky,30.6697,-81.4626,Hampton Inn Amelia Island at Fernandina Beach
10,39,Natal,BR,81.25,few clouds,-5.7950,-35.2094,Villa Park Hotel | Natal


In [17]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_vacation_df[["Lat", "Lng"]]

In [18]:
# 4a. Add a marker layer for each city to the map.
max_temp = clean_vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [33]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = clean_vacation_df.loc[clean_vacation_df["City"] == "Miraflores"]
vacation_end = clean_vacation_df.loc[clean_vacation_df["City"] == "Miraflores"]
vacation_stop1 = clean_vacation_df.loc[clean_vacation_df["City"] == "Acapulco"]
vacation_stop2 = clean_vacation_df.loc[clean_vacation_df["City"] == "Tecoanapa"] 
vacation_stop3 = clean_vacation_df.loc[clean_vacation_df["City"] == "Puerto Escondido"] 

In [34]:
vacation_start

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
8,36,Miraflores,MX,84.02,broken clouds,19.2167,-98.8083,Hotel Mansión


In [35]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = ((vacation_start.to_numpy()[0][5]), (vacation_start.to_numpy()[0][6]))
end = ((vacation_end.to_numpy()[0][5]), (vacation_end.to_numpy()[0][6]))
stop1 = ((vacation_stop1.to_numpy()[0][5]), (vacation_stop1.to_numpy()[0][6]))
stop2 = ((vacation_stop2.to_numpy()[0][5]), (vacation_stop2.to_numpy()[0][6]))
stop3 = ((vacation_stop3.to_numpy()[0][5]), (vacation_stop3.to_numpy()[0][6]))

In [40]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", 
#"BICYCLING", or "WALKING".
import gmaps.datasets
fig1 = gmaps.figure()
vacation_route = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], \
                                        travel_mode=('BICYCLING' or 'DRIVING' or 'WALKING'))
fig1.add_layer(vacation_route)
fig1

Figure(layout=FigureLayout(height='420px'))

In [41]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, 
                          vacation_end], ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,36,Miraflores,MX,84.02,broken clouds,19.2167,-98.8083,Hotel Mansión
1,155,Acapulco,MX,84.02,scattered clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
2,585,Tecoanapa,MX,83.52,broken clouds,16.5167,-98.7500,Hotel Manglares
3,387,Puerto Escondido,MX,86.83,clear sky,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
4,36,Miraflores,MX,84.02,broken clouds,19.2167,-98.8083,Hotel Mansión


In [43]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
itinerary_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map.
fig2 = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=itinerary_info)
fig2.add_layer(marker_layer)
# 11b. Display the figure
fig2

Figure(layout=FigureLayout(height='420px'))